<a href="https://colab.research.google.com/github/Rekt77/kisa_insuretech/blob/master/recommandation_webcrawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [355]:
# 한글 형태소 분석기 다운로드
!pip install konlpy

In [0]:
# 한국어 형태소분석기
import json
from konlpy.tag import Kkma
kor_nlp = Kkma()

ins_cat = ["운전자","상해","재물",]

# 우리의 불용어 사전
with open("/content/drive/My Drive/insuretech/nlp_stopword.json","r") as JSON:
  kor_stopwords=json.load(JSON)["KOR"]

In [0]:
import requests
from bs4 import BeautifulSoup

bobae_list = list()
bobae_test = list()

# 보배드림에서 문장을 긁어 온다 = 운전자
url = "https://www.bobaedream.co.kr/list"
myParams = {
    "code":"national",
    "or_gu":"10",
    "or_se":"desc",
    "pagescale":"30",
    "s_select":"Subject",
    "s_key":"%EB%B3%B4%ED%97%98",#보험
    "type":"list",
    "page": 1
}

res = requests.get(url)

for i in range(1,6):
  myParams["page"]=i
  res = requests.get(url)
  soup = BeautifulSoup(res.content,'lxml')
  titles = soup.find_all("a",class_="bsubject")
  for title in titles:
    if myParams["page"] == 5:
      bobae_test.append([title.text,"운전"])
    bobae_list.append([title.text,"운전"])


In [0]:
cho_list = list()
cho_test = list()

# 헬스조선에서 문장을 긁어온다 키워드 = 질병
url = 'http://health.chosun.com/list.html'

myParams={
    "nowcode":1,
    "pn":1
}

for i in range(1,6):
  myParams["pn"]=i
  res = requests.get(url,params=myParams)
  soup = BeautifulSoup(res.content,'lxml')
  article_div = soup.find_all("dl",class_="list_item")
  for each in article_div:
    article = each.find_all("a")
    tmp_url = 'http://health.chosun.com/' + article[1].attrs['href']
    res = requests.get(tmp_url)
    soup = BeautifulSoup(res.content,'lxml')
    title = soup.find("h1",id="news_title_text_id")
    if myParams["pn"] == 5:
      cho_test.append([title.text,"질병"])
    cho_list.append([title.text,"질병"])

In [0]:
dc_list = list()
dc_test = list()

# DC인사이드 보험갤러리 키워드 = 상해
url = "https://gall.dcinside.com/board/lists"

myParams = {"id":"insurance",
            "page":1,
            "s_type":"search_subject",
            "s_keyword":"%EC%83%81%ED%95%B4",#상해
            "board_type":"undefined"}

myHeader = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"
}

for i in range(1,6):
  myParams["page"] = i
  res = requests.get(url,params=myParams,headers=myHeader)
  soup = BeautifulSoup(res.text)
  for each in soup.find_all("td",class_="gall_tit ub-word")[1:]:
    if myParams["page"] == 5:
      dc_test.append([each.a.text,"상해"])
    dc_list.append([each.a.text,"상해"])

In [0]:
joint_list=bobae_list+cho_list+dc_list

In [367]:
# train_X = 문장 >> ('hi my name is', 'hi your name is')
# train_Y = 분류 >> ('운전','상해')
train_X,train_Y = list(zip(*joint_list))

# 토큰화
# 꼬꼬마 형태소 분석기를 이용해서 문장을 분석
# 형태소 단위로 분해하여 train_x에 저장

train_X = [kor_nlp.morphs(sentence) for sentence in train_X]
print(train_X)

[['기', '더기', '들', '바', '스', '러', '트', '려', '버리', '는', '멋지', 'ㄴ', '글'], ['호불호', '없', '는', 'BMW'], ['대한민국', '1', '이', 'ㄴ', '자', '인척', '하', '는', '새끼'], ['대한민국', '이', '재정비', '되', '었', '단', '날'], ['만화', ')', '후방', '주의'], ['ch', '2709', '실체'], ['한국', '이', '유독', '반일', '감정', '이', '심하', 'ㄴ', '이유', '.'], ['요즘', '보배'], ['나', '아', '경원', '의원', '이', '"', '민주당', '마음대로', '하라', '이', '고', '하', 'ㅂ니다', '.', '"'], ['홍', '콩', '시위', '현장', '에', '쓰', '어', '있', '는', '글'], ['저', '어', '보고', '직원', '2', '명', '을', '짜르', '라', '고하', '네요'], ['"', '윤미향', '·', '정의', '연', '보도', '는', '가짜', '뉴스', ',', '조선', '일보', '폐간', '운동', '돌입', '"'], ['이용수', '할머니', '옆', '에', '붙', '은', '그분'], ['정의', '연', '고발', '하', 'ㄴ', '단체', '의', '반전', 'ㅋㅋ'], ['위안부', '할머니', '들', '생활비', '끊', '은', '정부'], ['일본', '언론', '曰', ':', '한국', '잔인', '하', '고', '무섭', '다', '.'], ['박', '뿜계', '의원', '트윗', '...', '"', '절대', '과반', '177', '석', '"'], ['[', '보이스', '피', '싱', ']', '.', '사기', '의', '심', '.', '아버지', '의', '실종', '[', '꼭', '좀', '도와주', '세요', ']'], ['나이', '58', '살', '처먹'

In [0]:
# import string

# train_X_copy = train_X[:]
# train_X_clense = list()
# # 불용어 제거
# for sentence in train_X_copy:
#   tmp_sentence=list()
#   for token in sentence:
#     if token not in kor_stopwords and token not in string.punctuation:
#       tmp_sentence.append(token)
#   train_X_clense.append(tmp_sentence)

# train_X = train_X_clense[:]

In [317]:
train_X

[['기', '더기', '바', '스', '러', '트', '려', '버리', '는', '멋지', 'ㄴ', '글'],
 ['대한민국', '1', 'ㄴ', '인척', '는', '새끼'],
 ['만화', '후방', '주의'],
 ['ch', '2709', '실체'],
 ['한국', '유독', '반일', '감정', '심하', 'ㄴ', '이유'],
 ['요즘', '보배'],
 ['경원', '의원', '민주당', '하라', '고', 'ㅂ니다'],
 ['홍', '콩', '시위', '현장', '쓰', '있', '는', '글'],
 ['보고', '직원', '2', '명', '짜르', '라', '고하', '네요'],
 ['윤미향', '·', '정의', '연', '보도', '는', '가짜', '뉴스', '조선', '일보', '폐간', '운동', '돌입'],
 ['이용수', '할머니', '옆', '붙', '은', '그분'],
 ['정의', '연', '고발', 'ㄴ', '단체', '반전', 'ㅋㅋ'],
 ['위안부', '할머니', '생활비', '끊', '은', '정부'],
 ['일본', '언론', '曰', '한국', '잔인', '고', '무섭', '다'],
 ['박', '뿜계', '의원', '트윗', '...', '절대', '과반', '177', '석'],
 ['보이스', '피', '싱', '사기', '심', '아버지', '실종', '꼭', '도와주', '세요'],
 ['나이', '58', '살', '처먹', '는', '...'],
 ['日', '매체', '한국인', '무섭', 'ㄴ', '국민성', '비판'],
 ['최강', '욱', '소령', '시절', '4', '스타', '구속', '썰'],
 ['의류', '행거', '필요', '시', 'ㄴ', '분', '가져가', '세요'],
 ['솔직히', '문', '재인', '안', '좋아하', 'ㅁ'],
 ['최', '배근', '내가', '수요', '집회', '참석', 'ㄴ', '이유'],
 ['국', '게', '추천', '요정', '퇴

In [0]:
# 단어별 인덱스를 만들것임. 모르는 단어는 unknown
# 단어별 인덱스를 만드는 이유 == one-hot-encoding을 위해서
word2index={'unknown' : 0}

# 문장을 추출
for x in train_X:
  # 형태소단위로 분해된 토큰을 추출
    for token in x:
      #word2index에 추가되지 않은 단어이면 단어를 추가
        if word2index.get(token)==None:
          #추가 할때마다 word2index의 길이를 인덱스로 반영
            word2index[token]=len(word2index)

In [369]:
class2index = {'운전' : 0, '질병' : 1, '상해' : 2}
print(word2index)
print(class2index)

{'unknown': 0, '기': 1, '더기': 2, '들': 3, '바': 4, '스': 5, '러': 6, '트': 7, '려': 8, '버리': 9, '는': 10, '멋지': 11, 'ㄴ': 12, '글': 13, '호불호': 14, '없': 15, 'BMW': 16, '대한민국': 17, '1': 18, '이': 19, '자': 20, '인척': 21, '하': 22, '새끼': 23, '재정비': 24, '되': 25, '었': 26, '단': 27, '날': 28, '만화': 29, ')': 30, '후방': 31, '주의': 32, 'ch': 33, '2709': 34, '실체': 35, '한국': 36, '유독': 37, '반일': 38, '감정': 39, '심하': 40, '이유': 41, '.': 42, '요즘': 43, '보배': 44, '나': 45, '아': 46, '경원': 47, '의원': 48, '"': 49, '민주당': 50, '마음대로': 51, '하라': 52, '고': 53, 'ㅂ니다': 54, '홍': 55, '콩': 56, '시위': 57, '현장': 58, '에': 59, '쓰': 60, '어': 61, '있': 62, '저': 63, '보고': 64, '직원': 65, '2': 66, '명': 67, '을': 68, '짜르': 69, '라': 70, '고하': 71, '네요': 72, '윤미향': 73, '·': 74, '정의': 75, '연': 76, '보도': 77, '가짜': 78, '뉴스': 79, ',': 80, '조선': 81, '일보': 82, '폐간': 83, '운동': 84, '돌입': 85, '이용수': 86, '할머니': 87, '옆': 88, '붙': 89, '은': 90, '그분': 91, '고발': 92, '단체': 93, '의': 94, '반전': 95, 'ㅋㅋ': 96, '위안부': 97, '생활비': 98, '끊': 99, '정부': 100, '일본': 101, '언론': 102,

In [0]:
import torch.nn as nn
import torch
from torch import optim
from torch.autograd import Variable

In [0]:
# Bag of Words 만들기
def make_BoW(seq,word2index):
  #문장에 대한 원핫 인코딩 구현
  #보유한 단어 수 크기의 0으로 초기화된 텐서 구현
    tensor = torch.zeros(len(word2index))

    #seq는 형태소 문장
    #w는 나누어진 형태소
    for w in seq:
      # word2index로 부터 w의 인덱스를 구함
        index = word2index.get(w)
        # 인덱스가 있을 경우
        if index!=None:
          # 해당 인덱스의 값을 1로 만들어줌
            tensor[index]+=1.
        # 인덱스가 없는경우
        else:
          # unknown의 값이 인덱스가 되고
          # 해당 인덱스의 값을 1로 만들어줌
            index = word2index['unknown']
            tensor[index]+=1.
    return tensor

In [373]:
# 문장을 원핫 인코딩으로 만든다.
make_BoW(train_X[0],word2index)

1223

In [374]:
# ['운전', '을', '배우', '고', '있', '는', '중']의 인덱스가 0,1,2,3,4,5,6 이라면
# 0~6번째 요소가 1로 채워진 텐서가 만들어짐

train_X = torch.cat([Variable(make_BoW(x,word2index)).view(1,-1) for x in train_X])
train_Y = torch.cat([Variable(torch.LongTensor([class2index[y]])) for y in train_Y])

# 각각의 문장이 원핫인코딩 벡터로 나타나져 있는 Variable
train_X

# 각각의 문장에 대응되는 클래스가 나타나져 있는 Variable
train_Y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [375]:
len(train_X)

349

In [376]:
len(train_Y)

349

In [0]:
# BoW classifier 만들기
class BoWClassifier(nn.Module):
    def __init__(self,vocab_size,output_size):
        super(BoWClassifier,self).__init__()
        
        # 뉴런간 선형결합
        # 선형회귀 인풋 차원 : vocab_size
        # 선형회귀 아웃풋 차원 : output_size
        # vocab을 아웃풋에 연결하여 연산
        self.linear = nn.Linear(vocab_size,output_size)
    
    # 아웃풋을 리턴하는 함수 nn.module에서 오버라이팅
    def forward(self,inputs):
        return self.linear(inputs)

In [386]:
def accuracy(pred, y):
    #preds는 데이터가 학습되는 과정에서 라벨을 분류한 값
    preds = torch.argmax(pred, dim=1)
    return (preds == y).float().mean()

# 학습
epochs = 5
LR = 0.1

# 뉴런이 349개
# 인풋 크기는 word2index의 길이
# 아웃풋은 3(클래스가 3개이기 때문)
model = BoWClassifier(len(word2index),3)

# cost함수 선언
# crossentropyLoss
loss_function = nn.CrossEntropyLoss()

# 경사하강법으로 학습
optimizer = optim.SGD(model.parameters(),lr=LR)

for epoch in range(epochs):
    model.zero_grad()
    hypothesis = model(train_X)
    loss = loss_function(hypothesis,train_Y)
    print("epoch:",epoch+1,"loss:",loss.item(),"acc:",float(accuracy(hypothesis,train_Y))*100)
    
    loss.backward()
    optimizer.step()

epoch: 1 loss: 1.1153812408447266 acc: 20.057307183742523
epoch: 2 loss: 1.0878278017044067 acc: 39.54154849052429
epoch: 3 loss: 1.061510443687439 acc: 64.18337821960449
epoch: 4 loss: 1.0363712310791016 acc: 77.65042781829834
epoch: 5 loss: 1.012349009513855 acc: 85.10028719902039


In [0]:
test_set = bobae_test+cho_test+dc_test

In [388]:
test_set

[['기더기들 바스러트려버리는 멋진 글', '운전'],
 ['호불호 없는 BMW', '운전'],
 ['대한민국 1인자 인척 하는새끼', '운전'],
 ['대한민국이 재정비 되었단 날', '운전'],
 ['만화)   후방주의', '운전'],
 ['ch2709 실체', '운전'],
 ['한국이 유독 반일감정이 심한 이유.', '운전'],
 ['요즘 보배', '운전'],
 ['나경원 의원이 "민주당 마음대로 하라고 합니다."', '운전'],
 ['홍콩 시위현장에 써있는 글', '운전'],
 ['저보고 직원 2명을 짜르라고하네요', '운전'],
 ['"윤미향·정의연 보도는 가짜뉴스, 조선일보 폐간운동 돌입"', '운전'],
 ['이용수 할머니 옆에 붙은 그분', '운전'],
 ['정의연 고발한 단체의 반전 ㅋㅋ', '운전'],
 ['위안부 할머니들 생활비 끊은 정부', '운전'],
 ['일본 언론 曰 : 한국 잔인하고 무섭다.', '운전'],
 ['박뿜계 의원 트윗..."절대 과반 177석"', '운전'],
 ['[보이스피싱].사기의심.아버지의 실종[꼭좀 도와주세요]', '운전'],
 ['나이 58살 처먹는 동안...', '운전'],
 ['日매체 "한국인 무서운 국민성" 비판', '운전'],
 ['최강욱 소령시절 4스타 구속썰', '운전'],
 ['의류 행거 필요하신분 가져가세요', '운전'],
 ['나 솔직히 문재인 안 좋아함.', '운전'],
 ['최배근 "내가 수요집회에 참석한 이유"', '운전'],
 ['국게 추천요정 퇴근합니다.', '운전'],
 ['자꾸 누군가 신고해서 삭제되는 뉴스', '운전'],
 ['오늘 일본대사관 앞 수요집회 장면', '운전'],
 ['조폭 유튜버들과의 전쟁 선포', '운전'],
 ['(대박)MBC 또 터트렸다. 윤석열 장모 최씨 (개/ 검찰)..', '운전'],
 ['포항꽃집 사건내용입니다', '운전'],
 ["'보톡스 허가 취소' 최후 항변 나서는 메디톡스…미용피부외과학회 등 '선처' 호소", '질병'],
 ["식사에 '이것' 추

In [400]:
index2class = {v:k for k,v in class2index.items()}

acc = list()
for test in test_set:
    # 테스트 데이터 불러오기
    X = kor_nlp.morphs(test[0])
    X = Variable(make_BoW(X,word2index)).view(1,-1)
    
    pred = model(X)
    
    # 3가지 클래스 분류 중에 가장 높은것 추출
    pred = pred.max(1)[1].item()
    print("Input : %s" % test[0])
    print("Prediction : %s" % index2class[pred])
    print("Truth : %s" % test[1], end="")
    acc.append(index2class[pred]==test[1])

print("Accuracy:",acc.count(True)/len(acc)*100)



Input : 기더기들 바스러트려버리는 멋진 글
Prediction : 운전
Truth : 운전
Input : 호불호 없는 BMW
Prediction : 운전
Truth : 운전
Input : 대한민국 1인자 인척 하는새끼
Prediction : 운전
Truth : 운전
Input : 대한민국이 재정비 되었단 날
Prediction : 운전
Truth : 운전
Input : 만화)   후방주의
Prediction : 운전
Truth : 운전
Input : ch2709 실체
Prediction : 운전
Truth : 운전
Input : 한국이 유독 반일감정이 심한 이유.
Prediction : 운전
Truth : 운전
Input : 요즘 보배
Prediction : 운전
Truth : 운전
Input : 나경원 의원이 "민주당 마음대로 하라고 합니다."
Prediction : 운전
Truth : 운전
Input : 홍콩 시위현장에 써있는 글
Prediction : 운전
Truth : 운전
Input : 저보고 직원 2명을 짜르라고하네요
Prediction : 운전
Truth : 운전
Input : "윤미향·정의연 보도는 가짜뉴스, 조선일보 폐간운동 돌입"
Prediction : 운전
Truth : 운전
Input : 이용수 할머니 옆에 붙은 그분
Prediction : 운전
Truth : 운전
Input : 정의연 고발한 단체의 반전 ㅋㅋ
Prediction : 운전
Truth : 운전
Input : 위안부 할머니들 생활비 끊은 정부
Prediction : 운전
Truth : 운전
Input : 일본 언론 曰 : 한국 잔인하고 무섭다.
Prediction : 운전
Truth : 운전
Input : 박뿜계 의원 트윗..."절대 과반 177석"
Prediction : 운전
Truth : 운전
Input : [보이스피싱].사기의심.아버지의 실종[꼭좀 도와주세요]
Prediction : 운전
Truth : 운전
Input : 나이 58살 처먹는 동안...
Predict